# About this code

- Code author: Chenguang Wang   
- Email: c.wang-8@tudelft.nl; samwangchenguang@gmail.com
- Affiliation: Delft University of Technology
- Project name: Generating multivariate load states using a (conditional) variational autoencoder
- Motivation: This is a project for PSCC2022 – Power Systems Computation Conference: [Homepage of the conference](https://pscc2022.pt/)
- Aim of this code: Generating 35-dimensional load states using a variational autoencoder with output noise parameter co-optimised (Auto σ')
- A preprint is available, and you can check this paper for more details  [Link of the paper](https://arxiv.org/abs/2110.11435)
    - Paper authors: Chenguang Wang, Ensieh Sharifnia, Zhi Gao, Simon H. Tindemans, Peter Palensky
    - Accepted for publication at PSCC 2022 and a special issue of EPSR
    - If you use (parts of) this code, please cite the preprint or published paper

# Preparation (Library)

In [ ]:
# ==================== Preparation (library) =======================
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

# Preparation (Data)
- Input: Load data for 35 European countries between 2017 and 2018

In [ ]:
# ============================ Get all data ====================================

Data = pd.read_csv("../Data/17-18_35_conditioned.csv") # National_load_data_all
Train = Data # Note: Different from the other case study (CASE STUDY: EUROPEAN LOAD DATA), we use all data for training in this study (MULTI-AREA ADEQUACY ASSESSMENT).

In [ ]:
# ================= Separate load data and time label ==========================

# Get column location
Austria=Data.columns.get_loc('AL_load_actual_entsoe_power_statistics')
UK=Data.columns.get_loc('UK_UKM_load_actual_entsoe_power_statistics')
hour_sin=Data.columns.get_loc('hour_sin')
hour_cos=Data.columns.get_loc('hour_cos')

# Get only load data
Train_load=Data.iloc[:,Austria:(UK+1)]
Train_load.index=range(len(Train_load)) # index rearrange

# Get only time label
Train_label=Data.iloc[:,hour_sin:(hour_cos+1)]
Train_label.index=range(len(Train_label)) # index rearrange

In [ ]:
# ============= Dara processing for all training and testing data  =============
#============================= Traing data processing ==========================

# To get "normalized" training data
Tr = Train_load
max_Tr = np.max(Tr, axis = 0) # reserve the maximum value of basic training data
min_Tr = np.min(Tr, axis = 0) # reserve the minimum value of basic training data
Tr_nor = (Tr-min_Tr)/(max_Tr-min_Tr) # normalized training data

mean_Tr = np.mean(Tr, axis = 0) # Mean Value

min_Tr_astype = min_Tr.values.astype(np.float32) # Training 
max_Tr_astype = max_Tr.values.astype(np.float32) # Testing 
mean_Tr_astyped = mean_Tr.values.astype(np.float32) # Mean

# "Date format" conversion for all data
TR_load = Tr_nor.values.astype(np.float32) # Training Data                                                                                                                
TR_label = Train_label.values.astype(np.float32) # Training label

# Preparation (Neuro Network Dessign)

In [ ]:
# ================== Preparation (Neuro network dessign) =======================

# ========================== Initialization fuction ============================

def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / tf.sqrt(in_dim / 2.)
    return tf.random_normal(shape=size, stddev=xavier_stddev)

# ===================== Data dimension calcuation ==============================

# Dimension of the input
X_dim_all=np.shape(Train_load)
X_dim = X_dim_all[1]
# Dimension of the time label
y_dim_all =np.shape(TR_label) 
y_dim = y_dim_all[1]
# Dimension of the hidden layer
h_dim_1 = 32
# Dimension of the hidden layer
h_dim_2 = 24
# Dimension of the hidden layer
h_dim_3 = 16
# Dimension of the latent space
z_dim = 8

# =============================== Q(z|X,c) =====================================

# Data
X = tf.placeholder(tf.float32, shape=[None, X_dim])
z = tf.placeholder(tf.float32, shape=[None, z_dim])

# Layer definitions
Q_W1 = tf.Variable(xavier_init([X_dim, h_dim_1]))
Q_b1 = tf.Variable(tf.zeros(shape=[h_dim_1]))

Q_W2 = tf.Variable(xavier_init([h_dim_1, h_dim_2]))
Q_b2 = tf.Variable(tf.zeros(shape=[h_dim_2]))

Q_W3 = tf.Variable(xavier_init([h_dim_2, h_dim_3]))
Q_b3 = tf.Variable(tf.zeros(shape=[h_dim_3]))

Q_W4_mu = tf.Variable(xavier_init([h_dim_3, z_dim]))
Q_b4_mu = tf.Variable(tf.zeros(shape=[z_dim]))

Q_W4_sigma = tf.Variable(xavier_init([h_dim_3, z_dim]))
Q_b4_sigma = tf.Variable(tf.zeros(shape=[z_dim]))

# Network
def Q(X): 
    #inputs = tf.concat(axis=1, values=[X, c]) 
    h1 = tf.nn.relu(tf.matmul(X, Q_W1) + Q_b1)
    h2 = tf.nn.relu(tf.matmul(h1, Q_W2) + Q_b2)
    h3 = tf.nn.relu(tf.matmul(h2, Q_W3) + Q_b3)
    z_mu = tf.matmul(h3, Q_W4_mu) + Q_b4_mu
    z_logvar = tf.matmul(h3, Q_W4_sigma) + Q_b4_sigma
    return z_mu, z_logvar

# ================Sampling a z from N~(z_mu|tf.exp(z_logvar))===================

def sample_z(mu, log_var):
    eps = tf.random_normal(shape=tf.shape(mu))
    return mu + tf.exp(log_var / 2) * eps

# =============================== P(X|z) =======================================

# Layer definitions
P_W1 = tf.Variable(xavier_init([z_dim , h_dim_3]))  
P_b1 = tf.Variable(tf.zeros(shape=[h_dim_3]))

P_W2 = tf.Variable(xavier_init([h_dim_3 , h_dim_2]))  
P_b2 = tf.Variable(tf.zeros(shape=[h_dim_2]))

P_W3 = tf.Variable(xavier_init([h_dim_2 , h_dim_1]))  
P_b3 = tf.Variable(tf.zeros(shape=[h_dim_1]))

P_W4_mu = tf.Variable(xavier_init([h_dim_1, X_dim]))
P_b4_mu = tf.Variable(tf.zeros(shape=[X_dim]))

P_W4_logvar = tf.Variable(xavier_init([h_dim_1, X_dim]))
P_b4_logvar = tf.Variable(tf.zeros(shape=[X_dim]))


# Network
def P(z):
    h1 = tf.nn.relu(tf.matmul(z, P_W1) + P_b1) 
    h2 = tf.nn.relu(tf.matmul(h1, P_W2) + P_b2) 
    h3 = tf.nn.relu(tf.matmul(h2, P_W3) + P_b3) 
    Gen_mu = tf.matmul(h3, P_W4_mu) + P_b4_mu # Gen_mu : μ_x(z)
    Gen_logvar= tf.matmul(h3, P_W4_logvar) + P_b4_logvar # Gen_logvar : log((σ_x(z))**2)   
    eps_p = tf.random_normal(shape=tf.shape(Gen_mu)) # epsilon
    Gen_noisy = Gen_mu + tf.exp(Gen_logvar/2)*eps_p # Gen_noisy = μ_x(z) + σ_x(z)  * eps_p
    return Gen_mu, Gen_logvar , Gen_noisy

# Training (VAE; Auto σ')

In [ ]:
# ================================ Training ====================================

# Training condition: lr (learning rate)=0.0001; batch_size (mb_size) =64;
#                     iteration=200,000; σ' Auto

#Traning setting 
mb_size = 64 # Batch size
lr = 0.0001 #learning rate
β = 10 # weight ratio of kl_loss and recon_loss 
iteration = 200000 # define the iteration steps

# Training functions
z_mu, z_logvar = Q(X) # z follows N~(z_μ,z_log(σ^2)) 
z_sample = sample_z(z_mu, z_logvar) # Sampling z_sample from N~(z_μ, tf.exp(z_log(σ^2)))
Gen_mu_train, Gen_logvar_train, Gen_noisy_train = P(z_sample) # generate (hat) x_μ, x_log(σ^2), x given z_sample

# Generation functions
Gen_mu_sample, Gen_logvar_sample, Gen_noisy_sample = P(z) # Sampling (tilde) x_μ, x_log(σ^2), x given z follows N~(0|1)

# Training loss when σ' is automatically generated
recon_loss = 0.5 *tf.reduce_sum((Gen_mu_train - X)**2/tf.exp(Gen_logvar_train)+Gen_logvar_train, 1) # recon_error

kl_loss  =   β*0.5 * tf.reduce_sum(tf.exp(z_logvar) - 1. - z_logvar + z_mu**2, 1) # KL_error

vae_loss =   tf.reduce_mean(recon_loss + kl_loss)+X_dim/2*tf.log(2*(math.pi)) # VAE loss

# Monitor what the exact number 
recon_loss_mean = tf.reduce_mean(recon_loss)+X_dim/2*tf.log(2*(math.pi))

kl_loss_mean = tf.reduce_mean(kl_loss)

# Solver
solver = tf.train.AdamOptimizer(lr).minimize(vae_loss)

# Seesion
sess = tf.Session() # Session definition
sess.run(tf.global_variables_initializer())

for it in range(iteration): # Training process
    batch_index = np.random.choice(TR_load.shape[0], size=mb_size, replace=False)
    _, loss, KL_error, Recon_error = sess.run([solver, vae_loss, kl_loss_mean, recon_loss_mean], 
                                              feed_dict={X: TR_load[batch_index]})                                                                                                                    
    if it % 1000 == 0:
        print('Iter: {}'.format(it))
        print('Loss: {:.4}'. format(loss))
        print('KL_error: {}'. format(KL_error))
        print('Recon_error: {}'. format(Recon_error))
        print()
print("Training Finished!") 

#Save the reslut    
saver = tf.train.Saver() 
saver.save(sess, './filename.chkp') 

# Session Recover
sess = tf.Session()
saver = tf.train.Saver()                                                     
saver.restore(sess, 'filename.chkp')

# Load data Generation

In [ ]:
# ==========================Generate load data==================================

# =======================Function to recover data===============================

def Gen_recover_function(Gen_to_be_recovered): 
    Gen_recovered=Gen_to_be_recovered*(max_Tr_astype-min_Tr_astype)+min_Tr_astype
    return Gen_recovered

# How many years' data do you want to generate?
year=10

# ========================= Geneter load data ==================================

Gen_mu_new, Gen_logvar_new, Gen_noisy_new = sess.run([Gen_mu_sample, Gen_logvar_sample, Gen_noisy_sample], 
                                             feed_dict={z: np.random.randn(len(Train_load)*year,z_dim)})  
    
Gen_mu_all = Gen_mu_new   # All generated data mu
Gen_mu_recovered_all = Gen_recover_function(Gen_mu_new)   # All generated data mu (rescaled)
Gen_noisy_all = Gen_noisy_new # All generated data noisy
Gen_noisy_recovered_all = Gen_recover_function(Gen_noisy_new)   # All generated data noisy (rescaled)

# Explanation for the file names
- filename with "σ'_auto": output noise parameter is co-optimised during training (Auto σ')
- filename with "σ'_"+ a "number": output noise parameter is  fixed (Manually set σ')

- filename with "mu": generated data without noise (noise free)
- filename with "noisy": generated data with noise (noisy)

- filename with "Time_all": generated data for 24 hours
- filename with "Time_"+ a "number": generated data for a specific hour of the day

- filename with "nor": generated data that are not rescaled to the original scale of historical data
- filename without "nor": generated data that are rescaled to the original scale of historical data

- filename with a β number: data generated with that specific β number
- filename without a β number: the number of β is 1 by default

- filename with "VAE" specified: it is a VAE model
- filename without "VAE" specified: it is a CVAE model by default

- filename with "35" in the end: a model for 35-dimensional input
- filename without "35" in the end: a model for 32-dimensional input by default

In [ ]:
# Save data
pd.DataFrame(Gen_mu_recovered_all).to_csv("σ'_auto_Time_all_mu_VAE_β={:.1f}_35.csv".format(β))
pd.DataFrame(Gen_noisy_recovered_all).to_csv("σ'_auto_Time_all_noisy_VAE_β={:.1f}_35.csv".format(β))
pd.DataFrame(Gen_mu_all).to_csv("σ'_auto_Time_all_mu_nor_VAE_β={:.1f}_35.csv".format(β))
pd.DataFrame(Gen_noisy_all).to_csv("σ'_auto_Time_all_noisy_nor_VAE_β={:.1f}_35.csv".format(β))